## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
import os
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode

from contract_utils import execute_bash, instantiate_contract, execute_contract, query_contract


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 3
NUMBER_OF_ACTIVATED_PARTICIPANTS = 2
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [ipfs_client.add_str(_nickname + 'avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [1_000_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(20)
COEF_DOWN = str(5)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')
DISPLAY_TX_EXECUTION = False

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(40)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(25)
PASSPORT_CONTRACT_ADDRESS = 'bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/Users/sergenedashkovsky/Library/Python/3.8/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.10.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
claims_list = []
for i in range(NUMBER_OF_PARTICIPANTS):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
                  'nickname': NICKNAME_LIST[i],
                  'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{WALLET_ADDRESS}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_without_proof' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,1000000,john16474127780,QmZVpc2G8oV5ghDoWsVC8znfn1EjgiAvGrYS3KfNLymsrp,0x4878aae4c66be69d15c590dac467692337e5b0cf,0x2ca0986d548ab75ad252cbe701d7393df117468ba5e4...,bostrom1aw9qyjhmm8krtggpx728uutegazlq3lwajtkrk,cosmos1aw9qyjhmm8krtggpx728uutegazlq3lw7pl9a3,success clog skill length already armed entry ...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x6a1a572b734e7626e85cc33fccf5db6e3c7b8ccc05f3...,{'pub_key': 'AiPJ68b+MkLs9faOtLGNOOEK23SduB41Z...,eyJwdWJfa2V5IjoiQWlQSjY4YitNa0xzOWZhT3RMR05PT0...
1,1000000,john16474127781,QmX7VAaprn4xonSTt1d51K7Uf71ZfnkJzajgpdZ3HpE4oD,0x744443c73710d15eb9970ce7ebf470d36569176b,0x0d2abf8b07d08932a44f6fe95b729a6b3651f6d07616...,bostrom1z55mxwhr9j287hzt60y90mvueyrs3480uwent8,cosmos1z55mxwhr9j287hzt60y90mvueyrs3480ladq4q,village have emotion large august merry escape...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x7631668215a5b300662228dafa47f08d87c797a4ea09...,{'pub_key': 'AkkM9ngmwB05rEDTAaAUMz8C3oR1GqKXm...,eyJwdWJfa2V5IjoiQWtrTTluZ213QjA1ckVEVEFhQVVNej...
2,1000000,john16474127782,QmcCbZ86xHNaa2cgSvN5JmcFAvYxB9GdmggQUGPGxFBos6,0x15720b613d56c03d0a2a7f0690ff21fbcc03f9b4,0xbc5fce7dfbe0d75c0044109ba5fd5b9f8513b7603a31...,bostrom1f3wzccz56d8c48tuldchxf57878kx88wc8jhvh,cosmos1f3wzccz56d8c48tuldchxf57878kx88wm5xyjs,dress tobacco ignore cinnamon party isolate ca...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x342d548d44fd16875b7e91b2427a4aa33108f5633647...,{'pub_key': 'A0gTMu4ihLFsg/gDG7e5uAD5BLrmGUe6B...,eyJwdWJfa2V5IjoiQTBnVE11NGloTEZzZy9nREc3ZTV1QU...


## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [3]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph')
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph')
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6
Avatar subgraph contract address: bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq
Proof subgraph contract address: bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4


### Instantiate Passport Contract

In [4]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra


### Set executor in the Subgraph Contracts

In [5]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                            contract_address=subgraph_contract_address,
                            gas=600000,
                            display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [6]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1hh235cquh47tk7mj2hfkdz70g6m787qpx9vledlyz0n3j4jt0d8sgj92mq


In [7]:
def get_contract_name(contract_address: str) -> str:
    address_dict = {gift_contract_address: 'Gift Contract',
                    passport_contract_address: 'Passport Contract',
                    WALLET_ADDRESS: 'Passport Owner Address',
                    name_subgraph_contract_address: 'Name Subgraph Contract',
                    avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                    proof_subgraph_contract_address: 'Proof Subgraph Contract'}
    try:
        return address_dict[contract_address]
    except KeyError:
        return contract_address


def get_name_from_cid(ipfs_hash: str, row=None) -> str:
    if row is None:
        return ipfs_hash
    cid_name_dict = {
        row['avatar']: 'Avatar',
        ipfs_client.add_str(row['nickname']): 'Nickname',
        ipfs_client.add_str(row['ethereum_address']): 'Ethereum Address',
        ipfs_client.add_str(row['cosmos_address']): 'Cosmos Address',
        ipfs_client.add_str(WALLET_ADDRESS): 'Passport Owner Address'}
    try:
        return cid_name_dict[ipfs_hash]
    except KeyError:
        return ipfs_hash


def parse_contract_execution_json(contract_execution_json: str, row=None) -> None:
    print('\nEvents')
    if len(json.loads(contract_execution_json)['logs']) == 0:
        print(json.loads(contract_execution_json)['raw_log'])
    for log_item in json.loads(contract_execution_json)['logs']:
        for event_item in log_item['events']:
            print('')
            if event_item['type'] == 'message':
                if len(event_item["attributes"]) == 3:
                    print(f'message from {get_contract_name(event_item["attributes"][-1]["value"])} {event_item["attributes"][1]["value"]} {event_item["attributes"][0]["value"]}')
                else:
                    print(event_item)
            elif event_item['type'] == 'execute':
                print('execute')
                for attr_item in event_item["attributes"]:
                    if attr_item["key"] == '_contract_address':
                        print(f'\texecute contract: {get_contract_name(attr_item["value"])}')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'reply':
                print('reply')
                for attr_item in event_item["attributes"]:
                    if attr_item["key"] == '_contract_address':
                        print(f'\treply contract: {get_contract_name(attr_item["value"])}')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'cyberlink':
                print('cyberlinks')
                for i, attr_item in enumerate(event_item['attributes']):
                    if attr_item['key'] == 'particleFrom':
                        print(f'\t{get_name_from_cid(attr_item["value"], row=row)} -> {get_name_from_cid(event_item["attributes"][i + 1]["value"], row=row)}')
                    elif attr_item['key'] == 'particleTo':
                        pass
                    elif attr_item['key'] == 'neuron':
                        print(f'\tneuron: {get_contract_name(attr_item["value"])}\n')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'coin_received':
                print('coin received')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'coin_spent':
                print('coin spent')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'wasm':
                print('wasm')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'transfer':
                print('transfer')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            else:
                print(event_item)

### Create Merkle Root

In [8]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [9]:
root_source_list

[{'address': '0x4878aae4c66be69d15c590dac467692337e5b0cf',
  'amount': '1000000'},
 {'address': '0x744443c73710d15eb9970ce7ebf470d36569176b',
  'amount': '1000000'},
 {'address': '0x15720b613d56c03d0a2a7f0690ff21fbcc03f9b4',
  'amount': '1000000'},
 {'address': 'cosmos1aw9qyjhmm8krtggpx728uutegazlq3lw7pl9a3',
  'amount': '1000000'},
 {'address': 'cosmos1z55mxwhr9j287hzt60y90mvueyrs3480ladq4q',
  'amount': '1000000'},
 {'address': 'cosmos1f3wzccz56d8c48tuldchxf57878kx88wm5xyjs',
  'amount': '1000000'},
 {'address': '0xf2749114feaad68854e01c8ee762c7170532fdfd',
  'amount': '10000000'}]

In [10]:
execute_bash(f'rm {ROOT_FILE_NAME}')
root_output, root_error = execute_bash(
    f'merkle-airdrop-cli generateRoot --file {ROOT_SOURCE_FILE_NAME} >> {ROOT_FILE_NAME} && cat {ROOT_FILE_NAME}')
root = root_output.replace('\n', '')
print(f'Merkle root: {root}')

Merkle root: 725942cbb284688da040f146cb3441852460f6d8034896768a7f5c1e87f7d14a


In [11]:
root_register_output = execute_contract(execute_query=f'''{{"register_merkle_root":{{"merkle_root":"{root}"}}}}''',
                                        contract_address=gift_contract_address)
parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 725942cbb284688da040f146cb3441852460f6d8034896768a7f5c1e87f7d14a


#### Get Merkle Root form the Gift Contract

In [12]:
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

{'data': {'merkle_root': '725942cbb284688da040f146cb3441852460f6d8034896768a7f5c1e87f7d14a'}}


In [13]:
def generate_proof(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    _proof_output, _ = execute_bash(f'merkle-airdrop-cli generateProofs --file {ROOT_SOURCE_FILE_NAME} --address {claim_row[network + "_address"]} --amount {claim_row.amount}',
                                   display_data=display_data)
    return _proof_output.replace('\n','').replace(" ",'').replace("'", '"')

claims_df.loc[:, 'ethereum_proof'] = claims_df.apply(lambda x: generate_proof(x), axis=1)
claims_df.loc[:, 'cosmos_proof'] = claims_df.apply(lambda x: generate_proof(x, network='cosmos'), axis=1)
claims_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,1000000,john16474127780,QmZVpc2G8oV5ghDoWsVC8znfn1EjgiAvGrYS3KfNLymsrp,0x4878aae4c66be69d15c590dac467692337e5b0cf,0x2ca0986d548ab75ad252cbe701d7393df117468ba5e4...,bostrom1aw9qyjhmm8krtggpx728uutegazlq3lwajtkrk,cosmos1aw9qyjhmm8krtggpx728uutegazlq3lw7pl9a3,success clog skill length already armed entry ...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x6a1a572b734e7626e85cc33fccf5db6e3c7b8ccc05f3...,{'pub_key': 'AiPJ68b+MkLs9faOtLGNOOEK23SduB41Z...,eyJwdWJfa2V5IjoiQWlQSjY4YitNa0xzOWZhT3RMR05PT0...,"[""f164f48466bb32f66eabed8fbb52de5953bb1b4e756e...","[""0ea5afdcea05b78568ba41aa3b120c669db9fc111445..."
1,1000000,john16474127781,QmX7VAaprn4xonSTt1d51K7Uf71ZfnkJzajgpdZ3HpE4oD,0x744443c73710d15eb9970ce7ebf470d36569176b,0x0d2abf8b07d08932a44f6fe95b729a6b3651f6d07616...,bostrom1z55mxwhr9j287hzt60y90mvueyrs3480uwent8,cosmos1z55mxwhr9j287hzt60y90mvueyrs3480ladq4q,village have emotion large august merry escape...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x7631668215a5b300662228dafa47f08d87c797a4ea09...,{'pub_key': 'AkkM9ngmwB05rEDTAaAUMz8C3oR1GqKXm...,eyJwdWJfa2V5IjoiQWtrTTluZ213QjA1ckVEVEFhQVVNej...,"[""78900d7c06c43ae20970087421f80e49257815363f6a...","[""cf75dec9a3423593845301454490058e275aba2beb5d..."
2,1000000,john16474127782,QmcCbZ86xHNaa2cgSvN5JmcFAvYxB9GdmggQUGPGxFBos6,0x15720b613d56c03d0a2a7f0690ff21fbcc03f9b4,0xbc5fce7dfbe0d75c0044109ba5fd5b9f8513b7603a31...,bostrom1f3wzccz56d8c48tuldchxf57878kx88wc8jhvh,cosmos1f3wzccz56d8c48tuldchxf57878kx88wm5xyjs,dress tobacco ignore cinnamon party isolate ca...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x342d548d44fd16875b7e91b2427a4aa33108f5633647...,{'pub_key': 'A0gTMu4ihLFsg/gDG7e5uAD5BLrmGUe6B...,eyJwdWJfa2V5IjoiQTBnVE11NGloTEZzZy9nREc3ZTV1QU...,"[""3e3c9fa66f3317f6bfd46130ae15b61227730b12f820...","[""84daa30ce2e40d0ed2745dd96dff99e0fd3d42ada1fa..."


In [14]:
proof_output, _ = execute_bash(f'merkle-airdrop-cli generateProofs --file {ROOT_SOURCE_FILE_NAME} --address {"0xF2749114FeaAD68854E01C8eE762C7170532FdfD".lower()} --amount 10000000',
                                   display_data=False)
proof_output.replace('\n','').replace(" ",'').replace("'", '"')

'["82ff93c206404af75c9add59e997d3e2adb90f217c9532941a713ea6d851cacd","39b2d510f6915efa477b25c813366807b4cbea928649381f068eace67c643354","01e792d24b7dd735767c69605e131d51db5a770d21ff4f144f29424ef9a5b1f1"]'

In [15]:
"0xF2749114FeaAD68854E01C8eE762C7170532FdfD".lower()

'0xf2749114feaad68854e01c8ee762c7170532fdfd'

### Create Passports

In [16]:
def create_passport(claim_row: pd.Series, display_data: bool = False):
    return execute_contract(execute_query=f'''{{"create_passport":{{"avatar":"{claim_row["avatar"]}", "nickname":"{claim_row["nickname"]}"}}}}''',
                            contract_address=passport_contract_address,
                            gas=600000,
                            display_data=display_data)

for index, row in tqdm(claims_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        create_passport_json = create_passport(row, display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(create_passport_json, row=row)
    else:
        create_passport(row)

0it [00:00, ?it/s]


Events

cyberlinks
	Passport Owner Address -> Nickname
	Nickname -> Passport Owner Address
	neuron: Name Subgraph Contract

	Passport Owner Address -> Avatar
	Avatar -> Passport Owner Address
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 49


### Proof (Add) Address to Passports

In [17]:
def proof_address(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_contract(execute_query=f'''{{"proof_address":{{"address":"{claim_row[network + "_address"]}", "nickname":"{claim_row["nickname"]}", "signature":"{claim_row[network + "_message_signature"]}"}}}}''',
                            contract_address=passport_contract_address,
                            gas=400000,
                            display_data=display_data)

for index, row in tqdm(claims_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        proof_ethereum_address_json = proof_address(row, display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(proof_ethereum_address_json, row=row)

        proof_cosmos_address_json = proof_address(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(proof_cosmos_address_json, row=row)
    else:
        proof_address(row)
        proof_address(row, network='cosmos')

0it [00:00, ?it/s]


Events

cyberlinks
	Passport Owner Address -> Ethereum Address
	Ethereum Address -> Passport Owner Address
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16474127781
	address: 0x744443c73710d15eb9970ce7ebf470d36569176b

Events

cyberlinks
	Passport Owner Address -> Cosmos Address
	Cosmos Address -> Passport Owner Address
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16474127781
	address: cosmos1z55mxwhr9j287hzt60y90mvueyrs3480ladq4q


### Claim

In [18]:
def claim(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_contract(execute_query=f'''{{"claim":{{"nickname":"{claim_row['nickname']}", "gift_claiming_address":"{claim_row[network + "_address"]}", "gift_amount":"{claim_row.amount}", "proof":{claim_row[network + "_proof"]}}}}}''',
                            contract_address=gift_contract_address,
                            gas=400000,
                            display_data=display_data)

for index, row in tqdm(claims_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        claim_ethereum_json = claim(row, display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(claim_ethereum_json)

        claim_cosmos_json = claim(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(claim_cosmos_json)
    else:
        claim_ethereum_json_2 = claim(row)
        parse_contract_execution_json(claim_ethereum_json_2)

        claim_cosmos_json_2 = claim(row, network='cosmos')
        parse_contract_execution_json(claim_cosmos_json_2)

0it [00:00, ?it/s]


Events

coin received
	receiver: Passport Owner Address
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Passport Owner Address
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: 0x4878aae4c66be69d15c590dac467692337e5b0cf
	target: Passport Owner Address
	amount: 20000000

Events

coin received
	receiver: Passport Owner Address
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Passport Owner Address
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: cosmos1aw9qyjhmm8krtggpx728uutegazlq3lw7pl9a3
	target: Passport Owner Address
	amount: 2000

### Release Gift

In [19]:
def release(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_contract(execute_query=f'''{{"release":{{"gift_address":"{claim_row[network + "_address"]}"}}}}''',
                            contract_address=gift_contract_address,
                            gas=400000,
                            display_data=display_data)

for index, row in tqdm(claims_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        release_ethereum_json = release(row, display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(release_ethereum_json)

        release_cosmos_json = release(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(release_cosmos_json)
    else:
        release(row)
        release(row, network='cosmos')


0it [00:00, ?it/s]


Events

coin received
	receiver: Passport Owner Address
	amount: 1990000boot

coin spent
	spender: Gift Contract
	amount: 1990000boot

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Passport Owner Address
	sender: Gift Contract
	amount: 1990000boot

wasm
	_contract_address: Gift Contract
	action: release
	address: Passport Owner Address
	gift_address: 0x4878aae4c66be69d15c590dac467692337e5b0cf
	stage: 9
	amount: 1990000

Events

coin received
	receiver: Passport Owner Address
	amount: 1990000boot

coin spent
	spender: Gift Contract
	amount: 1990000boot

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Passport Owner Address
	sender: Gift Contract
	amount: 1990000boot

wasm
	_contract_address: Gift Contract
	action: release
	address: Passport Owner Address
	gift_address: cosmos1aw9qyjhmm8krtggpx728uut

## Passport NFT testing
### Transfer Passport

In [20]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"portid": {{"portid": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['portid']
get_passport_id(WALLET_ADDRESS)

50

In [21]:
def transfer_passport(claim_row: pd.Series, token_id: str,  to_address: str = '', display_data: bool = False):
    if to_address == '':
        to_address = claim_row['bostrom_address']
    return execute_contract(execute_query=f'''{{"transfer_nft":{{"recipient":"{to_address}", "token_id":"{token_id}"}}}}''',
                            contract_address=passport_contract_address,
                            gas=600000,
                            display_data=display_data)


for index, row in tqdm(claims_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        create_passport_json = transfer_passport(row, token_id=get_passport_id(WALLET_ADDRESS), display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(create_passport_json, row=row)
        break
    else:
        create_passport_json = transfer_passport(row, token_id=get_passport_id(WALLET_ADDRESS))

0it [00:00, ?it/s]


Events
failed to execute message; message index: 0: dispatch: submessages: dispatch: submessages: your cyberlink already exists


### Burn

In [22]:
def burn_passport(claim_row: pd.Series, token_id: str,  display_data: bool = False):
    return execute_contract(execute_query=f'''{{"burn":{{"token_id":"{token_id}"}}}}''',
                            contract_address=passport_contract_address,
                            gas=400000,
                            display_data=display_data)

for index, row in tqdm(claims_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        burn_passport_json = burn_passport(row, token_id=get_passport_id(WALLET_ADDRESS), display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(burn_passport_json, row=row)
        break
    else:
        create_passport_json = burn_passport(row, token_id=get_passport_id(WALLET_ADDRESS))

0it [00:00, ?it/s]


Events

cyberlinks
	QmTkBzFu3KPH8VKb1zmGkMzkWey6EkbQnPKLRU43fRhMDs -> QmRjw7ZB1SN3c7F6K921zfmQdmqAAFGnQjKQWJnhqB2TSH
	QmRjw7ZB1SN3c7F6K921zfmQdmqAAFGnQjKQWJnhqB2TSH -> QmTkBzFu3KPH8VKb1zmGkMzkWey6EkbQnPKLRU43fRhMDs
	neuron: Name Subgraph Contract

	QmTkBzFu3KPH8VKb1zmGkMzkWey6EkbQnPKLRU43fRhMDs -> QmX7VAaprn4xonSTt1d51K7Uf71ZfnkJzajgpdZ3HpE4oD
	QmX7VAaprn4xonSTt1d51K7Uf71ZfnkJzajgpdZ3HpE4oD -> QmTkBzFu3KPH8VKb1zmGkMzkWey6EkbQnPKLRU43fRhMDs
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: Passport Owner Address
	token_id: 50


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +